In [5]:
# !pip install agentpy

In [3]:
import agentpy as ap
import numpy as np
# Visualization
import matplotlib.pyplot as plt
import IPython
import math
import random
from scipy.stats import truncnorm

In [4]:
def normalize(v):
    """ Normalize a vector to length 2. """
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return math.sqrt(2) * v / norm

p = {
    'size': 201,
    'seed': 123,
    'steps': 80,
    'population': 500,
    'nevent': 100,
    'pertlucky': 0.5,
    'talentmean': 0.6,
    'talentsd': 0.1,
    'radius': 1,
    'intialcap': 10,
    'continuousspace': False, 
}

In [63]:
class individual(ap.Agent):
    """ An agent with a position and velocity in a continuous space """
    
    
    def setup(self):
        self.capital = self.p.intialcap
        
        myclip_a = 0
        myclip_b = 1
        a, b = (myclip_a - self.p.talentmean) / self.p.talentsd, (myclip_b - self.p.talentmean) / self.p.talentsd
        self.talent = truncnorm.rvs(a, b, size=1)[0]
    
    
#     def setup_talent(self):
#         myclip_a = 0
#         myclip_b = 1

#         a, b = (myclip_a - self.p.talentmean) / self.p.talentsd, (myclip_b - self.p.talentmean) / self.p.talentsd
#         self.talent = truncnorm.rvs(a, b, size=1)[0]
        
    
    def setup_pos(self, space):
        self.space = space
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
        
        
    def take_action(self):
        intersects = self.neighbors(self, distance=self.p.radius)
        for nb in intersects:
            if nb.type == "lucky_event":
                if self.talent > random.uniform(0, 1):
                    self.capital *= 2
#                     print("lucky: ", self.capital)
            if nb.type == "unlucky_event":
                self.capital /= 2
#                 print("unlucky: ", self.capital)
    
    def get_position(self, space):
        return space.positions[self]
        
        
class lucky_event(ap.Agent):
    
    def setup(self):
        # self.velocity = normalize(self.model.nprandom.random(2) - 0.5)
        pass
    
    def update_velocity(self):
        if self.p.continuousspace:
            self.velocity = normalize(self.model.nprandom.random(2) - 0.5)
        else:
            directions = [[0,2], [0,-2], [2,0], [-2,0]]
            self.velocity = np.array((random.choice(directions)))
            
    def setup_type(self):
        self.velocity
    
    def setup_pos(self, space):       
        self.space = space
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
        
    def update_position(self):
        self.space.move_by(self, self.velocity)
        
    def take_action(self):
        pass
        

class unlucky_event(ap.Agent):
    
    def setup(self):
        # self.velocity = normalize(self.model.nprandom.random(2) - 0.5)
        pass
    
    def update_velocity(self):
        if self.p.continuousspace:
            self.velocity = normalize(self.model.nprandom.random(2) - 0.5)
        else:
            directions = [[0,2], [0,-2], [2,0], [-2,0], [2, 2], [-2,-2], [2,-2], [-2, 2]]
            self.velocity = np.array((random.choice(directions)))
    
    def setup_pos(self, space):       
        self.space = space
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
    
    def update_position(self):
        self.space.move_by(self, self.velocity)
        
    def take_action(self):
        pass

class TvLModel(ap.Model):
    
    def setup(self):
        
        if self.p.continuousspace:
            self.space = ap.Space(self, shape=[self.p.size]*2, torus=True)
        else:
            self.space = ap.Grid(self, shape=[self.p.size]*2, torus=True)
        
        self.people = ap.AgentList(self, self.p.population, individual)
        self.space.add_agents(self.people, random=True)
        # self.people.setup_talent()
        # self.people.setup_capital()
        self.people.setup_pos(self.space)
        
        self.lucky_events = ap.AgentList(self, self.p.nevent, lucky_event)
        self.space.add_agents(self.lucky_events, random=True)
        self.lucky_events.setup_pos(self.space)

        self.unlucky_events = ap.AgentList(self, self.p.nevent, unlucky_event)
        self.space.add_agents(self.unlucky_events, random=True)
        self.unlucky_events.setup_pos(self.space)
        
    def step(self):
        self.lucky_events.update_velocity()
        self.unlucky_events.update_velocity()
        self.lucky_events.update_position()
        self.unlucky_events.update_position()
        self.people.take_action()
        self.people.record('capital')
        self.people.record('talent')

    def update(self):
        self.record('individual capital', self.people.capital)

    def end(self):
        self.people.record('capital')
        self.people.record('talent')


def animation_plot_single(m, ax):

    ax.set_title(f"Talent vs Luck Model t={m.t}\n https://github.com/hongwei-bayer/talent-vs-luck")
    ppl_pos = [pos for agent, pos in m.space.positions.items() if agent.type == 'individual']
    ppl_pos = np.array(list(ppl_pos)).T  # Transform
    
    lucky_pos = [pos for agent, pos in m.space.positions.items() if agent.type == 'lucky_event']
    lucky_pos = np.array(list(lucky_pos)).T  # Transform
    
    unlucky_pos = [pos for agent, pos in m.space.positions.items() if agent.type == 'unlucky_event']
    unlucky_pos = np.array(list(unlucky_pos)).T  # Transform
    
    ax.scatter(*ppl_pos, s=50, c='black', marker="x")
    ax.scatter(*lucky_pos, s=50, c='green', marker="o")
    ax.scatter(*unlucky_pos, s=50, c='red', marker="o")
    ax.set_xlim(0, 201)
    ax.set_ylim(0, 201)
    ax.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
    # ax.set_axis_off()

In [67]:
'''Animate'''
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection=None)
animation = ap.animate(TvLModel(p), fig, ax, animation_plot_single, steps=80)

html = IPython.display.HTML(animation.to_jshtml(fps=15))

In [72]:
'''display in python notebook'''
# html

'display in python notebook'

In [70]:
'''Animation html write to file'''
with open('html_file.html', 'w') as f:
    f.write(html.data)

In [42]:
'''Model Run and Data Analysis'''
model = TvLModel(p)
results = model.run()
data = results.variables.individual.reset_index()[['obj_id', 'capital', 'talent']]
data = data.set_index('obj_id')

Completed: 80 steps
Run time: 0:00:01.307006
Simulation finished


In [76]:
final_capital[final_capital.capital > 500]

,obj_id,capital,talent
1519,20,2560.0,0.611182
5759,73,655360.0,0.801241
7679,97,640.0,0.920543
10239,129,20480.0,0.923977
15119,190,5242880.0,1.695298
15199,191,5120.0,0.912821
17839,224,1280.0,1.265441
31359,393,640.0,1.304393
